# Module 4

## 1.Load dataset

In [226]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer

In [227]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

customerID  gender  SeniorCitizen  ... MonthlyCharges TotalCharges  Churn
0  7590-VHVEG  Female              0  ...          29.85        29.85     No
1  5575-GNVDE    Male              0  ...          56.95       1889.5     No
2  3668-QPYBK    Male              0  ...          53.85       108.15    Yes
3  7795-CFOCW    Male              0  ...          42.30      1840.75     No
4  9237-HQITU  Female              0  ...          70.70       151.65    Yes

[5 rows x 21 columns]

## 2.EDA

In [228]:
df.columns = df.columns.str.lower().str.replace(' ','_')

In [229]:
df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

In [230]:
df.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [231]:
categorical = list(df.dtypes[df.dtypes == 'object'].index)
numerical   =list(df.dtypes[df.dtypes != 'object'].index)

In [232]:
for col in categorical:
    df[col] = df[col].str.lower().str.replace(' ','_')
    

In [233]:
categorical.remove('customerid')
categorical.remove('churn')


In [234]:
df[categorical].describe()

gender partner  ... paperlessbilling     paymentmethod
count    7043    7043  ...             7043              7043
unique      2       2  ...                2                 4
top      male      no  ...              yes  electronic_check
freq     3555    3641  ...             4171              2365

[4 rows x 15 columns]

In [235]:
df['churn'] = df['churn'] == 'yes'

In [236]:
df.churn= df['churn'].astype(int)

In [237]:
df.head()

customerid  gender  seniorcitizen  ... monthlycharges totalcharges  churn
0  7590-vhveg  female              0  ...          29.85        29.85      0
1  5575-gnvde    male              0  ...          56.95      1889.50      0
2  3668-qpybk    male              0  ...          53.85       108.15      1
3  7795-cfocw    male              0  ...          42.30      1840.75      0
4  9237-hqitu  female              0  ...          70.70       151.65      1

[5 rows x 21 columns]

## 3.split the dataset

In [238]:
from sklearn.model_selection import train_test_split

In [239]:
full_train , test_set = train_test_split(df, test_size=0.2 , random_state=1)
train_set , val_set = train_test_split(full_train, test_size=0.25 , random_state=1)
len(train_set),len(val_set),len(test_set)

(4225, 1409, 1409)

In [240]:
train_set = train_set.reset_index(drop=True)
val_set   = val_set.reset_index(drop=True)
test_set  = test_set.reset_index(drop=True)

In [241]:
y_train = train_set['churn'].values
y_val   = val_set['churn'].values
y_test  = test_set['churn'].values


In [242]:
del train_set['churn']
del val_set['churn']
del test_set['churn']


## 4.train the model

In [243]:
numerical + categorical

['seniorcitizen',
 'tenure',
 'monthlycharges',
 'totalcharges',
 'gender',
 'partner',
 'dependents',
 'phoneservice',
 'multiplelines',
 'internetservice',
 'onlinesecurity',
 'onlinebackup',
 'deviceprotection',
 'techsupport',
 'streamingtv',
 'streamingmovies',
 'contract',
 'paperlessbilling',
 'paymentmethod']

In [244]:
dv = DictVectorizer(sparse=False)

In [245]:
train_dicts = train_set[categorical + numerical].to_dict(orient='records')

In [246]:
val_dicts = val_set[categorical + numerical].to_dict(orient='records')

In [247]:
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

In [248]:
model = LogisticRegression()

In [249]:
y_train.mean()

np.float64(0.2686390532544379)

In [250]:
model.fit(X_train,y_train)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [258]:
y_pred = model.predict_proba(X_val)[: , 1]

In [263]:
accuracy = ((y_pred> 0.5) == y_val).mean()

In [264]:
accuracy

np.float64(0.8034066713981547)